In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import statsmodels.formula.api as smf
import mglearn

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import urllib.request

#Импортируем нужные библиотеки:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn import cross_validation
from sklearn import metrics 
from pandas import DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Kobe Bryant  shot selection

Cсылка на соревнование: https://www.kaggle.com/c/kobe-bryant-shot-selection

Goal: Fun and education

Using 20 years of data on Kobe's swishes and misses, can you predict which shots will find the bottom of the net? This competition is well suited for practicing classification basics, feature engineering, and time series analysis. Practice got Kobe an eight-figure contract and 5 championship rings. What will it get you?

This data contains the location and circumstances of every field goal attempted by Kobe Bryant took during his 20-year career. Your task is to predict whether the basket went in (shot_made_flag).

We have removed 5000 of the shot_made_flags (represented as missing values in the csv file). These are the test set shots for which you must submit a prediction. You are provided a sample submission file with the correct shot_ids needed for a valid prediction.

In [97]:
data = pd.read_csv('/Users/stanislav/Downloads/FE 3-2/data/kobe.csv')

In [98]:
data.shot_made_flag.dropna()

1        0.0
2        1.0
3        0.0
4        1.0
5        0.0
6        1.0
8        1.0
9        0.0
10       0.0
11       1.0
12       1.0
13       0.0
14       0.0
15       0.0
17       1.0
18       0.0
20       0.0
21       0.0
22       1.0
23       1.0
24       1.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
30       1.0
31       0.0
38       1.0
39       1.0
        ... 
30661    1.0
30662    0.0
30663    0.0
30665    0.0
30666    0.0
30667    0.0
30669    0.0
30670    0.0
30671    0.0
30672    1.0
30673    0.0
30674    1.0
30675    0.0
30676    1.0
30677    0.0
30678    0.0
30679    0.0
30681    1.0
30683    0.0
30684    0.0
30685    0.0
30687    0.0
30688    1.0
30689    0.0
30690    0.0
30691    0.0
30692    0.0
30694    1.0
30695    0.0
30696    0.0
Name: shot_made_flag, Length: 25697, dtype: float64

In [99]:
data.dropna(subset=['shot_made_flag'], inplace=True)

In [101]:
data=data.drop(['game_date'],1)

In [102]:
data=pd.get_dummies(data)

In [103]:
data_feat=data.drop('shot_made_flag',1)

In [104]:
data_feat.head()

,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,...,opponent_PHI,opponent_PHX,opponent_POR,opponent_SAC,opponent_SAS,opponent_SEA,opponent_TOR,opponent_UTA,opponent_VAN,opponent_WAS
1,12,20000012,34.0443,-157,0,-118.4268,10,1,0,22,...,0,0,1,0,0,0,0,0,0,0
2,35,20000012,33.9093,-101,135,-118.3708,7,1,0,45,...,0,0,1,0,0,0,0,0,0,0
3,43,20000012,33.8693,138,175,-118.1318,6,1,0,52,...,0,0,1,0,0,0,0,0,0,0
4,155,20000012,34.0443,0,0,-118.2698,6,2,0,19,...,0,0,1,0,0,0,0,0,0,0
5,244,20000012,34.0553,-145,-11,-118.4148,9,3,0,32,...,0,0,1,0,0,0,0,0,0,0


**Задания:**

1. Провести анализ данных. Много хороших примеров анализа можно посмотреть здесь https://www.kaggle.com/c/kobe-bryant-shot-selection/kernels
2. Подготовить фичи для обучения модели - нагенерить признаков, обработать пропущенные значения, проверить на возможные выбросы, обработать категориальные признаки и др.
3. Обучить линейную модель, Lasso, Ridge на тех же признаках - построить сравнительную таблицу коэффициентов, сделать заключения о том, как меняется величина коэффициентов, какие зануляются. Посчитать RSS

**Дополнительно**
4. Сравнить результаты на тестовом наборе данных - сделать train_test_split в самом начале, подготовить переменные, сравнить результаты работы классификаторов (те же 3), метрика ROC AUC
5. Построить PCA на подготовленных признаках, посмотреть, какие компоненты составляют наибольшую часть дисперсии целевой переменной

In [105]:
data.shot_made_flag.value_counts()

0.0    14232
1.0    11465
Name: shot_made_flag, dtype: int64

In [106]:
data_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25697 entries, 1 to 30696
Columns: 222 entries, game_event_id to opponent_WAS
dtypes: float64(2), int64(11), uint8(209)
memory usage: 9.1 MB


In [111]:
from sklearn.metrics import roc_auc_score
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(data_feat, data[['shot_made_flag']], test_size = 0.3, random_state = 0)
sgd = linear_model.SGDClassifier(alpha=0.001, n_iter=100, random_state = 0)
sgd.fit(train_data, train_labels)
model_predictions = sgd.predict(test_data)
print (metrics.accuracy_score(test_labels, model_predictions))
print (metrics.classification_report(test_labels, model_predictions))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.4417639429312581
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      4304
        1.0       0.44      1.00      0.61      3406

avg / total       0.20      0.44      0.27      7710



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [127]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=700, random_state=2)
forest.fit(train_data, train_labels)
model_predictions = forest.predict(test_data)
print (metrics.accuracy_score(test_labels, model_predictions))
print (metrics.classification_report(test_labels, model_predictions))

forest_auc = roc_auc_score(test_labels, forest.predict_proba(test_data)[:, 1])
print("AUC для случайного леса: {:.3f}".format(forest_auc))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.6683527885862516
             precision    recall  f1-score   support

        0.0       0.66      0.82      0.73      4304
        1.0       0.68      0.48      0.56      3406

avg / total       0.67      0.67      0.66      7710

AUC для случайного леса: 0.687


In [113]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_data, train_labels)
model_predictions = lr.predict(test_data)
print (metrics.accuracy_score(test_labels, model_predictions))
print (metrics.classification_report(test_labels, model_predictions))

0.5582360570687419
             precision    recall  f1-score   support

        0.0       0.56      1.00      0.72      4304
        1.0       0.00      0.00      0.00      3406

avg / total       0.31      0.56      0.40      7710



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [114]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(train_data, train_labels)
model_predictions = knn.predict(test_data)
print (metrics.accuracy_score(test_labels, model_predictions))
print (metrics.classification_report(test_labels, model_predictions))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.553307392996109
             precision    recall  f1-score   support

        0.0       0.58      0.70      0.64      4304
        1.0       0.49      0.37      0.42      3406

avg / total       0.54      0.55      0.54      7710



In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25697 entries, 1 to 30696
Columns: 223 entries, game_event_id to opponent_WAS
dtypes: float64(3), int64(11), uint8(209)
memory usage: 9.3 MB


In [93]:
data_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25697 entries, 1 to 30696
Columns: 1780 entries, game_event_id to opponent_WAS
dtypes: float64(2), int64(11), uint8(1767)
memory usage: 46.0 MB


In [ ]:
# лучшие показатели-у случайного леса